In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [77]:
df = pd.read_csv("insulin_dosage_prediction.csv")
df.drop(['patient_id'],axis=1,inplace=True)
df.columns

Index(['gender', 'age', 'family_history', 'glucose_level', 'physical_activity',
       'food_intake', 'previous_medications', 'BMI', 'HbA1c', 'weight',
       'insulin_sensitivity', 'sleep_hours', 'creatinine', 'Insulin'],
      dtype='object')

In [78]:
df.head()

,gender,age,family_history,glucose_level,physical_activity,food_intake,previous_medications,BMI,HbA1c,weight,insulin_sensitivity,sleep_hours,creatinine,Insulin
0,male,68,yes,103.49,9.28,high,none,33.87,7.42,54.77,1.49,6.07,0.63,steady
1,female,57,yes,113.35,6.67,high,insulin,19.10,11.66,114.93,1.35,8.60,1.08,up
2,male,24,yes,127.40,4.14,medium,oral,22.48,5.72,59.07,0.96,5.19,1.36,up
3,male,49,yes,138.79,6.80,medium,oral,31.67,11.91,50.81,0.61,4.79,1.23,up
4,male,65,no,128.42,4.01,low,insulin,28.50,5.72,65.95,1.16,7.68,0.62,steady


In [79]:
gender_label =LabelEncoder()
df['gender']=gender_label.fit_transform(df['gender'])
family_history_label =LabelEncoder()
df['family_history']=family_history_label.fit_transform(df['family_history'])
Insulin_label =LabelEncoder()
df['Insulin']=Insulin_label.fit_transform(df['Insulin'])
food_intake_label =LabelEncoder()
df['food_intake']=food_intake_label.fit_transform(df['food_intake'])
previous_medications_label =LabelEncoder()
df['previous_medications']=previous_medications_label.fit_transform(df['previous_medications'])

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                10000 non-null  int32  
 1   age                   10000 non-null  int64  
 2   family_history        10000 non-null  int32  
 3   glucose_level         10000 non-null  float64
 4   physical_activity     10000 non-null  float64
 5   food_intake           10000 non-null  int32  
 6   previous_medications  10000 non-null  int32  
 7   BMI                   10000 non-null  float64
 8   HbA1c                 10000 non-null  float64
 9   weight                10000 non-null  float64
 10  insulin_sensitivity   10000 non-null  float64
 11  sleep_hours           10000 non-null  float64
 12  creatinine            10000 non-null  float64
 13  Insulin               10000 non-null  int32  
dtypes: float64(8), int32(5), int64(1)
memory usage: 898.6 KB


In [81]:
# ---- SAVE ALL ENCODERS ----
with open("gender_label.pkl", "wb") as f:
    pickle.dump(gender_label, f)

with open("family_history_label.pkl", "wb") as f:
    pickle.dump(family_history_label, f)

with open("Insulin_label.pkl", "wb") as f:
    pickle.dump(Insulin_label, f)

with open("food_intake_label.pkl", "wb") as f:
    pickle.dump(food_intake_label, f)

with open("previous_medications.pkl", "wb") as f:
    pickle.dump(previous_medications_label, f)

In [82]:
x = df.drop('glucose_level', axis=1)
y = df['glucose_level']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

In [83]:
x.columns

Index(['gender', 'age', 'family_history', 'physical_activity', 'food_intake',
       'previous_medications', 'BMI', 'HbA1c', 'weight', 'insulin_sensitivity',
       'sleep_hours', 'creatinine', 'Insulin'],
      dtype='object')

In [84]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
# Transform x_test using the same scaler
x_test_scaled = scaler.transform(x_test)

In [85]:
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# ANN

In [86]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [87]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

C:\Users\nithi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [88]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

In [89]:
opt =tf.keras.optimizers.Adam(learning_rate=.01)
loss =tf.keras.losses.BinaryCrossentropy()

In [90]:
# Compile the model
model.compile(optimizer=opt,loss = "mse", metrics=['mse'])

In [91]:
# Set the Tensorboard

log_dir = 'logs/fit/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensor_flow_callback =TensorBoard(log_dir=log_dir, histogram_freq=1)

In [92]:
# Set Up Early Stopping 
early_stopping_callback =  EarlyStopping(monitor='val_loss', patience=50,  restore_best_weights=True)

In [93]:
history = model.fit(
    x_train_scaled, y_train,
    validation_data=(x_test_scaled, y_test),
    epochs=1000,
    callbacks=[tensor_flow_callback, early_stopping_callback])

Epoch 1/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 3111.5117 - mse: 3111.5117 - val_loss: 1610.6816 - val_mse: 1610.6816
Epoch 2/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1498.3969 - mse: 1498.3969 - val_loss: 1505.4911 - val_mse: 1505.4911
Epoch 3/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1466.3665 - mse: 1466.3665 - val_loss: 1511.1222 - val_mse: 1511.1222
Epoch 4/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1460.3170 - mse: 1460.3170 - val_loss: 1499.7516 - val_mse: 1499.7516
Epoch 5/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1443.6887 - mse: 1443.6887 - val_loss: 1462.8541 - val_mse: 1462.8541
Epoch 6/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1437.1698 - mse: 1437.1698 - val_loss: 1471.5226 - val_mse: 1471.5226
Epoch 7/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1439.7694 - mse: 1439.7694 - val_loss: 1478.0887 - val_mse: 1478.0887
Epoch 8/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1436.7786 - ms

In [94]:
model.save('reg_model.h5')

In [62]:
# Load Tensorboard Extension
#%load_ext tensorboard

In [63]:
#%tensorboard --logdir  logs/fit

In [95]:
test_loss, test_mse=model.evaluate(x_test_scaled,y_test)
print(test_loss, test_mse)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1389.3044 - mse: 1389.3044
1389.304443359375 1389.304443359375


In [124]:
df_ = pd.read_csv("insulin_dosage_prediction.csv")

dict_ = {}
for col in df_.columns[1:]:   # skip first column, usually index/ID
    dict_[col] = df_[col].iloc[100]   # <-- FIXED (NO [0])

op = dict_.pop('glucose_level')  # remove insulin label from input
print(dict_,op)


{'gender': 'male', 'age': 65, 'family_history': 'no', 'physical_activity': 0.49, 'food_intake': 'low', 'previous_medications': 'insulin', 'BMI': 28.82, 'HbA1c': 8.99, 'weight': 59.02, 'insulin_sensitivity': 0.55, 'sleep_hours': 5.66, 'creatinine': 1.29, 'Insulin': 'up'} 174.02


In [125]:
gender_label.classes_

array(['female', 'male'], dtype=object)

In [126]:
import warnings
from tensorflow.keras.models import load_model
import pickle
import numpy as np

# Load model safely
model = load_model("reg_model.h5", compile=False)

# Recompile with proper regression loss/metrics
model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=["mean_absolute_error"]
)

# Load encoders
with open('gender_label.pkl','rb') as file:
    gender_label = pickle.load(file)

with open('family_history_label.pkl','rb') as file:
    family_history_label = pickle.load(file)

with open('Insulin_label.pkl','rb') as file:
    Insulin_label = pickle.load(file)

with open('food_intake_label.pkl','rb') as file:
    food_intake_one = pickle.load(file)

with open('previous_medications.pkl','rb') as file:
    previous_medications = pickle.load(file)

# Label encoding
gender_enc = gender_label.transform([dict_['gender']])[0]
family_history_enc = family_history_label.transform([dict_['family_history']])[0]
food_intake_enc = food_intake_one.transform([dict_['food_intake']])[0]
previous_medications_enc = previous_medications.transform([dict_['previous_medications']])[0]
Insulin_label_enc = Insulin_label.transform([dict_['Insulin']])[0]

print("Gender Encoded:", gender_enc)
print("Family History Encoded:", family_history_enc)
print("Food Intake Encoded:", food_intake_enc)
print("Previous Medications Encoded:", previous_medications_enc)
print("Insulin :", Insulin_label_enc)


Gender Encoded: 1
Family History Encoded: 0
Food Intake Encoded: 1
Previous Medications Encoded: 1
Insulin : 3


In [127]:
dict_['gender'] = gender_enc
dict_['family_history'] = family_history_enc
dict_['food_intake'] = food_intake_enc
dict_['previous_medications'] = previous_medications_enc
dict_['Insulin'] = Insulin_label_enc

In [128]:
dict_

{'gender': 1,
 'age': 65,
 'family_history': 0,
 'physical_activity': 0.49,
 'food_intake': 1,
 'previous_medications': 1,
 'BMI': 28.82,
 'HbA1c': 8.99,
 'weight': 59.02,
 'insulin_sensitivity': 0.55,
 'sleep_hours': 5.66,
 'creatinine': 1.29,
 'Insulin': 3}

In [129]:
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
scaler

StandardScaler()

In [130]:
# Prediction 
# Convert to numpy array in correct order
X = np.array(list(dict_.values())).reshape(1, -1)
X_scaled = scaler.transform(X)
# Prediction
pred = model.predict(X_scaled)
print((pred[0][0]))
#0.267092

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
146.8327


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [464]:

import warnings

warnings.filterwarnings("ignore", message="X does not have valid feature names*")

df_ = pd.read_csv("insulin_dosage_prediction.csv")

for i in range(20):
    dict_ = {}
    for col in df_.columns[1:]:   # skip first column, usually index/ID
        dict_[col] = df_[col].iloc[i]   # <-- FIXED (NO [0])
    
    op = dict_.pop('Insulin')  # remove insulin label from input
    print(dict_,op)
    
    # Label encoding
    gender_enc = gender_label.transform([dict_['gender']])[0]
    family_history_enc = family_history_label.transform([dict_['family_history']])[0]
    food_intake_enc = food_intake_one.transform([dict_['food_intake']])[0]
    previous_medications_enc = previous_medications.transform([dict_['previous_medications']])[0]

    dict_['gender'] = gender_enc
    dict_['family_history'] = family_history_enc
    dict_['food_intake'] = food_intake_enc
    dict_['previous_medications'] = previous_medications_enc

    #print(dict_)

    # Prediction 
    # Convert to numpy array in correct order
    X = np.array(list(dict_.values())).reshape(1, -1)
    X_scaled = scaler.transform(X)
    # Prediction
    pred = model.predict(X_scaled)
    
    pred_class = np.argmax(pred, axis=1)
    print("Predicted class index:", Insulin_label.classes_[pred_class[0]])    #0.267092



{'gender': 'male', 'age': 68, 'family_history': 'yes', 'glucose_level': 103.49, 'physical_activity': 9.28, 'food_intake': 'high', 'previous_medications': 'none', 'BMI': 33.87, 'HbA1c': 7.42, 'weight': 54.77, 'insulin_sensitivity': 1.49, 'sleep_hours': 6.07, 'creatinine': 0.63} steady
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted class index: steady
{'gender': 'female', 'age': 57, 'family_history': 'yes', 'glucose_level': 113.35, 'physical_activity': 6.67, 'food_intake': 'high', 'previous_medications': 'insulin', 'BMI': 19.1, 'HbA1c': 11.66, 'weight': 114.93, 'insulin_sensitivity': 1.35, 'sleep_hours': 8.6, 'creatinine': 1.08} up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted class index: up
{'gender': 'male', 'age': 24, 'family_history': 'yes', 'glucose_level': 127.4, 'physical_activity': 4.14, 'food_intake': 'medium', 'previous_medications': 'oral', 'BMI': 22.48, 'HbA1c': 5.72, 'weight': 59.07, 'insulin_sensitivity': 0.96, 'sleep_hours': 5.19, 'creatinine': 1.36} up
1/1 ━━━━━━━━━━━

In [457]:
with open('Insulin_label.pkl','rb') as file:
    Insulin_label = pickle.load(file)
Insulin_label.classes_

array(['down', 'no', 'steady', 'up'], dtype=object)